In [2]:
import sqlite3
import pandas as pd

## Sentiment change analysis + Response time analysis 

In [3]:
conn_tweets = sqlite3.connect('tweets_airlines.db')

In [4]:
query_tweets = '''SELECT tweet_id, user_id, reply_to_user, conversation_id, conversation_level, compound
                  FROM TWEETS
                  '''
df_tweets = pd.read_sql(query_tweets, conn_tweets)

In [5]:
df_tweets = df_tweets[df_tweets['conversation_id'] != 'level > 1']

In [6]:
airlines_dict = {56377143: 'KLM', 106062176: 'AirFrance', 124476322: 'Lufthansa', 18332190: 'British_Airways'}

In [7]:
df_real = df_tweets[df_tweets['conversation_level'] > 1] 

In [8]:
convo_dict = {}

def conv_airline(airline_id):
    
    contact = df_real[(df_real['reply_to_user'] == airline_id) | (df_real['user_id'] == airline_id)]['conversation_id'].to_list()

    conversations = []
    
    for conv_id in contact: 
        try: 
            if conv_id not in conversations: 
                conversations.append(conv_id)
        except: 
            pass 
    
    convo_dict[airline_id] = conversations

In [9]:
def all_ids(airline_id: int):
    
    conversations_airline = {}
    
    for i in convo_dict[airline_id]:
        tweets = []
        root = float(i) // 1
        root = int(root)
        root = str(root) + '.0'
        tweets.extend(df_tweets[df_tweets['conversation_id'] == root]['tweet_id'].tolist())
        tweets.extend(df_tweets[df_tweets['conversation_id'] == i]['tweet_id'].tolist())
             
        conversations_airline[i] = tweets
    
    print(airline_id)
    return conversations_airline

In [10]:
def resolve(airline_id, tweet_ids): 
    df_resolve = df_tweets[df_tweets['tweet_id'].isin(tweet_ids) & (df_tweets['user_id'] != airline_id)]
    try:
        first_sentiment = df_resolve[df_resolve['conversation_level'] == df_resolve['conversation_level'].min()]['compound'].values[0]
        last_sentiment = df_resolve[df_resolve['conversation_level'] == df_resolve['conversation_level'].max()]['compound'].values[0]
        if first_sentiment != last_sentiment: 
            delta_sentiment = last_sentiment - first_sentiment
            return delta_sentiment
    except: 
        pass

In [11]:
for key, value in airlines_dict.items():
    conv_airline(key)

In [12]:
for key, value in airlines_dict.items():
    globals()[value] = all_ids(key)

56377143
106062176
124476322
18332190


In [13]:
change_klm = []
for key, value in KLM.items():
    sentiment = resolve(56377143, value)
    if sentiment:
        change_klm.append(sentiment)

In [14]:
change_airf = []
for key, value in AirFrance.items():
    sentiment = resolve(106062176, value)
    if sentiment:
        change_airf.append(sentiment)

In [15]:
change_luft = []
for key, value in Lufthansa.items():
    sentiment = resolve(124476322, value)
    if sentiment:
        change_luft.append(sentiment)

In [16]:
change_ba = []
for key, value in British_Airways.items():
    sentiment = resolve(18332190, value)
    if sentiment:
        change_ba.append(sentiment)

In [17]:
dict_airlines = {'British_Airways': change_ba, 'Lufthansa': change_luft, 'AirFrance': change_airf, 'KLM': change_klm}

In [30]:
df_klm = pd.DataFrame(change_klm)
df_klm.to_csv('klm.csv')

df_airf = pd.DataFrame(change_airf)
df_airf.to_csv('airfrance.csv')

df_ba = pd.DataFrame(change_ba)
df_ba.to_csv('british_air.csv')

df_luft = pd.DataFrame(change_luft)
df_luft.to_csv('lufthansa.csv')

In [194]:
#All tweets
query = '''
select *
from tweets
'''
df_all = pd.read_sql(query, conn_tweets)

In [31]:
#AIRLINES ID, AIRLINE NAME, AVERAGE RESPONSE TIME IN HOURS, NUMBER OF REPLIES, NUMBER OF TWEERS WITHOUT ROOT
56377143: 'KLM' #0.8719111713339356, number of replies: 5535, no root: 169
106062176: 'AirFrance' #2.458694745031473, number of replies: 1571, no root: 22
18332190: 'British_Airways' #3.655695285601354, number of replies: 17007, no root: 217
22536055: 'AmericanAir' #0.9004331833196957, number of replies: 21998, no root: 164
124476322: 'Lufthansa' #0.3443861483725179, number of replies: 2311, no root: 3
26223583: 'AirBerlin' #no tweets, number of replies: 0, no root: 0  
2182373406: 'AirBerlin assist' #no tweets, number of replies: 0, no root: 0  
38676903: 'easyJet' #1.9478903030746348, number of replies: 9114, no root: 46
1542862735: 'RyanAir' #6.48837453933432, number of replies: 3829, no root: 14
253340062: 'SingaporeAir' #1.674717692251939, number of replies: 1533, no root: 13
218730857: 'Qantas' #1.4089490321136848, number of replies: 1986, no root: 4
45621423: 'EtihadAirways' #0.39753437383872164, number of replies: 299, no root: 1
20626359: 'VirginAtlantic' #0.5191655092592593, number of replies: 2880, no root: 57

SyntaxError: illegal target for annotation (<ipython-input-31-e7c600c01ca4>, line 3)

In [198]:
def average_reply_time(airline_id):
    '''
    Calculates average response time of specific airline.
    :param airline_id: tweeter id of an airline.
    return: average response time in hours, number of responses and number of airline responses without root
    '''
    #All airline replies
    replies_airlines = '''
    select *
    from tweets
    where reply_to_tweet <> 0 and user_id = {}
    '''.format(airline_id)
    airline_replies = pd.read_sql(replies_airlines, conn_tweets)
    
    
    
    #Average response time in hours
    sum1 = 0
    #how many times root  was found
    d1 = 0
    #no root, time here could not be calculated
    d2 = 0
    for index, row in airline_replies.iterrows():
        #root tweet
        root = row['reply_to_tweet']
        root_tweet = df_all[df_all['tweet_id']==root]
        #sum of time of all replies
        if not root_tweet.empty:
            #when root tweet was created
            for i in root_tweet['created_at']:
                x = i
            sum1 = sum1 + (row['created_at'] - x)
            #number of iterations required to calculate average
            d1 += 1
        else:
            d2 += 1
    airline_average = (sum1/d1)/3600
    return (airline_average, d1, d2)

In [199]:
average_reply_time(106062176)

(2.458694745031473, 1571, 22)